In [1]:
!pip install --quiet optuna

     |████████████████████████████████| 348 kB 30.4 MB/s 
     |████████████████████████████████| 81 kB 8.2 MB/s 
     |████████████████████████████████| 209 kB 76.0 MB/s 
     |████████████████████████████████| 78 kB 6.3 MB/s 
     |████████████████████████████████| 147 kB 67.2 MB/s 
     |████████████████████████████████| 49 kB 6.3 MB/s 
     |████████████████████████████████| 112 kB 71.8 MB/s 


In [2]:
import numpy as np
import pandas as pd
import optuna as opt
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold, cross_val_score
from pathlib import Path
import gc

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
RANDOM_SEED = 2
DATA_DIR = Path("/content/drive/MyDrive/CS760")

In [5]:
df_train = pd.read_parquet(DATA_DIR/"18_train_main.parquet.snappy")
df_test = pd.read_parquet(DATA_DIR/"18_test_main.parquet.snappy")

print(f"Shape of the training data : {df_train.shape}")
print(f"Shape of the test data : {df_test.shape}")

Shape of the training data : (400000, 20)
Shape of the test data : (100000, 20)


In [6]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
r_id,400000.0,3.520570e+06,2.014568e+06,1.900000e+01,1.807999e+06,3.481960e+06,5.279578e+06,6.990279e+06
r_stars,400000.0,3.612058e+00,1.538455e+00,1.000000e+00,2.000000e+00,4.000000e+00,5.000000e+00,5.000000e+00
r_stars_square,400000.0,1.541380e+01,9.620302e+00,1.000000e+00,4.000000e+00,1.600000e+01,2.500000e+01,2.500000e+01
r_length,400000.0,1.331285e+02,1.146588e+02,2.000000e+00,5.700000e+01,1.010000e+02,1.710000e+02,1.049000e+03
u_friends_count,400000.0,1.888460e+02,5.627721e+02,1.000000e+00,2.000000e+00,4.000000e+01,1.680000e+02,1.499500e+04
u_review_count,400000.0,1.904740e+02,4.836991e+02,1.000000e+00,1.100000e+01,4.400000e+01,1.760000e+02,1.747300e+04
u_month_age,400000.0,4.279213e+01,3.461348e+01,3.802649e-07,1.360590e+01,3.649717e+01,6.519433e+01,2.017248e+02
u_comp_avg,400000.0,5.472043e-01,3.116161e+00,0.000000e+00,0.000000e+00,1.357143e-01,4.050633e-01,5.991333e+02
u_n_elite_yrs,400000.0,1.885272e+00,3.077123e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,1.600000e+01
u_fans,400000.0,2.476457e+01,1.290454e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.100000e+01,1.249700e+04


In [7]:
X_train, y_train = df_train.drop(['r_useful', 'r_id'], axis=1).values, df_train['r_useful'].values
X_test, y_test = df_test.drop(['r_useful', 'r_id'], axis=1).values, df_test['r_useful'].values

In [8]:
def objective(trial):
  """ Function to tune parameters """
  gc.collect()
  params = {
      "n_estimators":trial.suggest_int('n_estimators', 1, 1001, step=50),
      "max_depth" : trial.suggest_int("max_depth", 2, 20),
      "learning_rate" : trial.suggest_float('learning_rate', 0.001, 0.5, log=True),
      "subsample ": trial.suggest_float("subsample", 0.4, 1, step=0.1),
      "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1, step=0.1),
      "gamma": trial.suggest_float("gamma", 0, 1),
      "min_child_weight": trial.suggest_int("min_child_weight", 1, 31, step=2)
  }

  print("Currently running with:")
  print(params)

  model = xgb.XGBRegressor(objective="reg:squarederror",
                            n_jobs=-1,
                            grow_policy='lossguide',
                            tree_method="gpu_hist",
                            predictor="gpu_predictor",
                            booster='gbtree',
                            sampling_method='gradient_based',
                            eval_metrics=['rmse'],
                            random_state=RANDOM_SEED,
                            enable_categorical=False,
                            **params)
  pipe = make_pipeline(SimpleImputer(), StandardScaler(), model)

  scores = cross_val_score(pipe, X_train, y_train,
    scoring="neg_root_mean_squared_error", cv=KFold(3))

  return -scores.mean()

study = opt.create_study(direction='minimize', sampler=TPESampler(seed=RANDOM_SEED))
study.optimize(objective, n_trials=50)
print("Final best parameters:")
study.best_params

[I 2022-09-30 04:13:59,330] A new study created in memory with name: no-name-55b1087f-a8cc-4105-8a8f-64840851838f


Currently running with:
{'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.030445460008040798, 'subsample ': 0.7000000000000001, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.3303348210038741, 'min_child_weight': 7}


[I 2022-09-30 04:14:09,836] Trial 0 finished with value: 3.8465217851905407 and parameters: {'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.030445460008040798, 'subsample': 0.7000000000000001, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.3303348210038741, 'min_child_weight': 7}. Best is trial 0 with value: 3.8465217851905407.


Currently running with:
{'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.005249994057689375, 'subsample ': 0.8, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.13457994534493356, 'min_child_weight': 17}


[I 2022-09-30 04:14:34,624] Trial 1 finished with value: 3.5997811895650393 and parameters: {'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.005249994057689375, 'subsample': 0.8, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.13457994534493356, 'min_child_weight': 17}. Best is trial 1 with value: 3.5997811895650393.


...

[I 2022-09-30 07:12:23,060] Trial 48 finished with value: 3.386170700499368 and parameters: {'n_estimators': 1001, 'max_depth': 7, 'learning_rate': 0.003228687011848442, 'subsample': 1.0, 'colsample_bytree': 0.9, 'gamma': 0.07157263807679537, 'min_child_weight': 1}. Best is trial 42 with value: 3.3214255922607183.


Currently running with:
{'n_estimators': 1001, 'max_depth': 7, 'learning_rate': 0.0015442507231669593, 'subsample ': 1.0, 'colsample_bytree': 0.9, 'gamma': 0.08395599063040049, 'min_child_weight': 1}


[I 2022-09-30 07:13:00,769] Trial 49 finished with value: 3.6619988574252704 and parameters: {'n_estimators': 1001, 'max_depth': 7, 'learning_rate': 0.0015442507231669593, 'subsample': 1.0, 'colsample_bytree': 0.9, 'gamma': 0.08395599063040049, 'min_child_weight': 1}. Best is trial 42 with value: 3.3214255922607183.


Final best parameters:


{'n_estimators': 1001,
 'max_depth': 10,
 'learning_rate': 0.004110476315640756,
 'subsample': 0.9,
 'colsample_bytree': 0.8,
 'gamma': 0.12798176163484867,
 'min_child_weight': 1}

In [12]:
best_params = study.best_params.copy()

model = xgb.XGBRegressor(objective="reg:squarederror",
                          n_jobs=-1,
                          grow_policy='lossguide',
                          tree_method="gpu_hist",
                          predictor="gpu_predictor",
                          booster='gbtree',
                          sampling_method='gradient_based',
                          eval_metrics=['rmse'],
                          random_state=RANDOM_SEED,
                          enable_categorical=False,
                          **best_params)
pipe = make_pipeline(SimpleImputer(), StandardScaler(), model)
pipe.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('xgbregressor',
                 XGBRegressor(colsample_bytree=0.8, enable_categorical=False,
                              eval_metrics=['rmse'], gamma=0.12798176163484867,
                              grow_policy='lossguide',
                              learning_rate=0.004110476315640756, max_depth=10,
                              n_estimators=1001, n_jobs=-1,
                              objective='reg:squarederror',
                              predictor='gpu_predictor', random_state=2,
                              sampling_method='gradient_based', subsample=0.9,
                              tree_method='gpu_hist'))])

In [13]:
train_pred = pipe.predict(X_train)
test_pred = pipe.predict(X_test)
print(f"train results - RMSE: {mean_squared_error(y_train, train_pred, squared=False)}, MAE: {mean_absolute_error(y_train, train_pred)}")
print(f"test results - RMSE: {mean_squared_error(y_test, test_pred, squared=False)}, MAE: {mean_absolute_error(y_test, test_pred)}")

train results - RMSE: 2.2427276327197547, MAE: 1.1850129781585932
test results - RMSE: 3.1378709233663677, MAE: 1.2826775820058585


In [11]:
mean_train = np.mean(y_train)
print(f"Baseline mean model - test RMSE: {np.sqrt(np.mean((y_test - mean_train)**2))}")
print(f"Baseline mean model - test MAE: {np.mean(np.abs(y_test - mean_train))}")

Baseline mean model - test RMSE: 4.1185891526597125
Baseline mean model - test MAE: 1.8970029502499997
Baseline mean model - test RMSE: 4.1185891526597125
Baseline mean model - test MAE: 1.8970029502499997
